In [60]:
import pandas as pd
import numpy as np
import os

from sklearn.utils import all_estimators

estimators = all_estimators(type_filter='classifier')

def getClassifiers():
    all_clfs_names = []
    all_clfs_clf = []
    for name, ClassifierClass in estimators:
        try:
            clf = ClassifierClass()
            all_clfs_names.append(clf)
            all_clfs_clf.append(clf)
            print("{'name':'" + name + "', 'params': [[{}],[{}]]},")
        except Exception as e:
            continue
            # print('Unable to import', name)
            # print(e)
    return all_clfs_names, all_clfs_clf

df = pd.read_csv(r'data/data_processed.csv')
df_continous = pd.read_csv(r'data/data_continous.csv')
df_categorical = pd.read_csv(r'data/data_categorical.csv')

# --------------------------------------------------------------------------------------------------
# copying job satisfaction into new array
JSat = df['JobSatisfaction'].values

# splitting inputs by row index
# continous data
df_training_continous = df_continous.iloc[:1200,:]
df_validation_continous = df_continous.iloc[1200:,:]
# categorical data
df_training_categorical = df_categorical.iloc[:1200,:]
df_validation_categorical = df_categorical.iloc[1200:,:]
# splitting outputs by number
JSat_training = JSat[:1200]
JSat_validation = JSat[1200:]
# --------------------------------------------------------------------------------------------------

all_clfs_names, all_clfs_clf = getClassifiers()
estimators_trimmed = zip(all_clfs_names, all_clfs_names)

{'name':'AdaBoostClassifier', 'params': [[{}],[{}]]},
{'name':'BaggingClassifier', 'params': [[{}],[{}]]},
{'name':'BernoulliNB', 'params': [[{}],[{}]]},
{'name':'CalibratedClassifierCV', 'params': [[{}],[{}]]},
{'name':'CategoricalNB', 'params': [[{}],[{}]]},
{'name':'ComplementNB', 'params': [[{}],[{}]]},
{'name':'DecisionTreeClassifier', 'params': [[{}],[{}]]},
{'name':'DummyClassifier', 'params': [[{}],[{}]]},
{'name':'ExtraTreeClassifier', 'params': [[{}],[{}]]},
{'name':'ExtraTreesClassifier', 'params': [[{}],[{}]]},
{'name':'GaussianNB', 'params': [[{}],[{}]]},
{'name':'GaussianProcessClassifier', 'params': [[{}],[{}]]},
{'name':'GradientBoostingClassifier', 'params': [[{}],[{}]]},
{'name':'HistGradientBoostingClassifier', 'params': [[{}],[{}]]},
{'name':'KNeighborsClassifier', 'params': [[{}],[{}]]},
{'name':'LabelPropagation', 'params': [[{}],[{}]]},
{'name':'LabelSpreading', 'params': [[{}],[{}]]},
{'name':'LinearDiscriminantAnalysis', 'params': [[{}],[{}]]},
{'name':'LinearS

In [77]:
# define possible parameters for all models, this is like a config file :)
all_possible_model_params = [
{'name':'AdaBoostClassifier', 'params': [[{"n_estimators":100}, {"n_estimators":50}, {"n_estimators":200}],[{"random_state":None}],[{"learning_rate":0.1}, {"learning_rate":1}, {"learning_rate":10}, {"learning_rate":100}]]}, # can implement estimator = any decisiion tree classifier with params... default = DecisionTreeClassifier(max_depth=1)
{'name':'BaggingClassifier', 'params': [[{"n_estimators":100}, {"n_estimators":50}, {"n_estimators":200}, {"n_estimators":10}, {"n_estimators":5}],[{"max_samples": 1.0},{"max_samples": 2.0},{"max_samples": 0.1},{"max_samples": 0.5},{"max_samples": 1},{"max_samples": 10}],[{"max_features": 1.0},{"max_features": 2.0},{"max_features": 2.5},{"max_features": 10}],[{"bootstrap": False}, {"bootstrap": True}]]}, # same as above
{'name':'BernoulliNB', 'params': [[{"alpha": 1.0}, {"alpha": 100.0}, {"alpha": 1e-12}, {"alpha": 1e-6}, {"alpha": 1e-3}],[{"force_alpha": True}, {"force_alpha": False}], [{"binarize": None}, {"binarize": 0.0}, {"binarize": 1.0}, {"binarize": 2.5}, {"binarize": 5.0}], [{"fit_prior": True}, {"fit_prior": False}]]},
{'name':'CalibratedClassifierCV', 'params': [[{'method':'sigmoid'}, {'method:':'isotonic'}] ,[{"cv": None}, {"cv": 1}, {"cv": 3}, {"cv": 10}]]},
{'name':'CategoricalNB', 'params': [[{"fit_prior": True}, {"fit_prior": False}],[{"alpha": 0}, {"alpha": 1}, {"alpha": 3}, {"alpha": 5}]]},
{'name':'ComplementNB', 'params': [[{}],[{}]]},
{'name':'DecisionTreeClassifier', 'params': [[{}],[{}]]},
{'name':'DummyClassifier', 'params': [[{}],[{}]]},
{'name':'ExtraTreeClassifier', 'params': [[{}],[{}]]},
{'name':'ExtraTreesClassifier', 'params': [[{}],[{}]]},
{'name':'GaussianNB', 'params': [[{}],[{}]]},
{'name':'GaussianProcessClassifier', 'params': [[{}],[{}]]},
{'name':'GradientBoostingClassifier', 'params': [[{}],[{}]]},
{'name':'HistGradientBoostingClassifier', 'params': [[{}],[{}]]},
{'name':'KNeighborsClassifier', 'params': [[{}],[{}]]},
{'name':'LabelPropagation', 'params': [[{}],[{}]]},
{'name':'LabelSpreading', 'params': [[{}],[{}]]},
{'name':'LinearDiscriminantAnalysis', 'params': [[{}],[{}]]},
{'name':'LinearSVC', 'params': [[{}],[{}]]},
{'name':'LogisticRegression', 'params': [[{"C":0.01}, {"C":0.1}, {"C":1}, {"C":10}],[{"solver":'lbfgs'}, {"solver":'liblinear'}, {"solver":'newton-cg'}, {"solver":'sag'}, {"solver":'saga'}]]},
{'name':'LogisticRegressionCV', 'params': [[{}],[{}]]},
{'name':'MLPClassifier', 'params': [[{}],[{}]]},
{'name':'MultinomialNB', 'params': [[{}],[{}]]},
{'name':'NearestCentroid', 'params': [[{}],[{}]]},
{'name':'NuSVC', 'params': [[{}],[{}]]},
{'name':'PassiveAggressiveClassifier', 'params': [[{}],[{}]]},
{'name':'Perceptron', 'params': [[{}],[{}]]},
{'name':'QuadraticDiscriminantAnalysis', 'params': [[{}],[{}]]},
{'name':'RadiusNeighborsClassifier', 'params': [[{}],[{}]]},
{'name':'RandomForestClassifier', 'params': [[{}],[{}]]},
{'name':'RidgeClassifier', 'params': [[{}],[{}]]},
{'name':'RidgeClassifierCV', 'params': [[{}],[{}]]},
{'name':'SGDClassifier', 'params': [[{}],[{}]]},
{'name':'SVC', 'params': [[{"C":0.01}, {"C":0.1}, {"C":1}, {"C":10}],[{"kernel":"linear"}, {"kernel":"rbf"}, {"kernel":"sigmoid"}, {"kernel":"poly"}], [{"degree":1}, {"degree":2}, {"degree":3}, {"degree":4}]]}
]

In [78]:
# pair all possible parameters of a type with others of a different types... basically combinations of n by k from the parameters
import itertools

for model_family in all_possible_model_params:
    params = model_family["params"]
    combinations = []
    combinations = [p for p in itertools.product(*params)]
    print(combinations)

[({'n_estimators': 100}, {'random_state': None}, {'learning_rate': 0.1}), ({'n_estimators': 100}, {'random_state': None}, {'learning_rate': 1}), ({'n_estimators': 100}, {'random_state': None}, {'learning_rate': 10}), ({'n_estimators': 100}, {'random_state': None}, {'learning_rate': 100}), ({'n_estimators': 50}, {'random_state': None}, {'learning_rate': 0.1}), ({'n_estimators': 50}, {'random_state': None}, {'learning_rate': 1}), ({'n_estimators': 50}, {'random_state': None}, {'learning_rate': 10}), ({'n_estimators': 50}, {'random_state': None}, {'learning_rate': 100}), ({'n_estimators': 200}, {'random_state': None}, {'learning_rate': 0.1}), ({'n_estimators': 200}, {'random_state': None}, {'learning_rate': 1}), ({'n_estimators': 200}, {'random_state': None}, {'learning_rate': 10}), ({'n_estimators': 200}, {'random_state': None}, {'learning_rate': 100})]
[({'n_estimators': 100}, {'max_samples': 1.0}, {'max_features': 1.0}, {'bootstrap': False}), ({'n_estimators': 100}, {'max_samples': 1.0

In [53]:


model_classes = []

for clf_name, clf in estimators_trimmed:
    clf_params = []
    clf_name = str(clf_name)
    if(clf_name == "LogisticRegression"): clf_params = [{"C":0.01}, {"C":0.1}, {"C":1}, {"C":10}, {"solver":'lbfgs'}, {"solver":'liblinear'}, {"solver":'newton-cg'}, {"solver":'sag'}, {"solver":'saga'}]
    elif(clf_name == "DecisionTreeClassifier"):  clf_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
    elif(clf_name == "RandomForestClassifier"): clf_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
    elif(clf_name == "KNeighborsClassifier"): clf_params = [{"n_neighbors":3}, {"n_neighbors":5}]
    elif(clf_name == "GaussianNB"): clf_params = [{"var_smoothing":1e-9}, {"var_smoothing":1e-5}, {"var_smoothing":1e-12}]
    elif(clf_name == "SVC"): clf_params = [{"C":0.01}, {"C":0.1}, {"C":1}, {"C":10}, {"kernel":"linear"}, {"kernel":"rbf"}, {"kernel":"sigmoid"}, {"kernel":"poly"}]
    elif(clf_name == "AdaBoostClassifier"): clf_params = [{"n_estimators":100}, {"n_estimators":50}, {"n_estimators":200}, {"random_state":0}]
    elif(clf_name == "BaggingClassifier"): clf_params = [{}]
    elif(clf_name == 'BernoulliNB'): clf_params = [{}]
    elif(clf_name == 'CalibratedClassifierCV'): clf_params = [{}]
    elif(clf_name == 'CategoricalNB'): clf_params = [{"min_categories":17}] # doesn't work for some reason
    elif(clf_name == 'ClassifierChain'): clf_params = [{}]
    elif(clf_name == 'ComplementNB'): clf_params = [{}]
    elif(clf_name == 'DummyClassifier'): clf_params = [{}]
    elif(clf_name == 'ExtraTreeClassifier'): clf_params = [{}]
    elif(clf_name == 'ExtraTreesClassifier'): clf_params = [{}]
    elif(clf_name == 'GaussianProcessClassifier'): clf_params = [{}]
    elif(clf_name == 'GradientBoostingClassifier'): clf_params = [{}]
    elif(clf_name == 'HistGradientBoostingClassifier'): clf_params = [{}]
    elif(clf_name == 'LabelPropagation'): clf_params = [{}]
    elif(clf_name == 'LabelSpreading'): clf_params = [{}]
    elif(clf_name == 'LinearDiscriminantAnalysis'): clf_params = [{}]
    elif(clf_name == 'LinearSVC'): clf_params = [{}]
    elif(clf_name == 'LogisticRegressionCV'): clf_params = [{}]
    elif(clf_name == 'MLPClassifier'): clf_params = [{}]
    elif(clf_name == 'MultiOutputClassifier'): clf_params = [{}]
    elif(clf_name == 'MultinomialNB'): clf_params = [{}]
    elif(clf_name == 'NearestCentroid'): clf_params = [{}]
    elif(clf_name == 'NuSVC'): clf_params = [{}]
    elif(clf_name == 'OneVsOneClassifier'): clf_params = [{}]
    elif(clf_name == 'OneVsRestClassifier'): clf_params = [{}]
    elif(clf_name == 'OutputCodeClassifier'): clf_params = [{}]
    elif(clf_name == 'PassiveAggressiveClassifier'): clf_params = [{}]
    elif(clf_name == 'Perceptron'): clf_params = [{}]
    elif(clf_name == 'QuadraticDiscriminantAnalysis'): clf_params = [{}]
    elif(clf_name == 'RadiusNeighborsClassifier'): clf_params = [{}]
    elif(clf_name == 'RidgeClassifier'): clf_params = [{}]
    elif(clf_name == 'RidgeClassifierCV'): clf_params = [{}]
    elif(clf_name == 'SGDClassifier'): clf_params = [{}]
    elif(clf_name == 'StackingClassifier'): clf_params = [{}]
    elif(clf_name == 'VotingClassifier'): clf_params = [{}]
    
    model_classes.append([clf, clf_name, clf_params])
    print("Appended " + str(clf_name) + " with params " + str(clf_params))

insights = []
for Model, modelname, params_list in model_classes:
    # skip these because we don't know how to make them work.. :)
    if modelname == "CategoricalNB" or modelname == "ClassifierChain" or modelname == "MultiOutputClassifier" or modelname == "OneVsOneClassifier" or modelname == "OneVsRestClassifier" or modelname == "MultiOutputClassifier" or modelname == "OutputCodeClassifier" or modelname == "RadiusNeighborsClassifier" or modelname == "StackingClassifier" or modelname == "VotingClassifier":
        continue

    # check that params are initialized
    if params_list != []:
        for params in params_list:
            print(str(modelname) + " with x_length: " + str((df_validation_categorical).shape) + " y_length: " + str((JSat_validation).shape) + " with params: " + str(params))
            model = Model(**params)
            model.fit(df_training_categorical, JSat_training)
            score = model.score(df_validation_categorical, JSat_validation)
            insights.append((modelname, model, params, score))



AppendedAdaBoostClassifier() with params []
AppendedBaggingClassifier() with params []
AppendedBernoulliNB() with params []
AppendedCalibratedClassifierCV() with params []
AppendedCategoricalNB() with params []
AppendedComplementNB() with params []
AppendedDecisionTreeClassifier() with params []
AppendedDummyClassifier() with params []
AppendedExtraTreeClassifier() with params []
AppendedExtraTreesClassifier() with params []
AppendedGaussianNB() with params []
AppendedGaussianProcessClassifier() with params []
AppendedGradientBoostingClassifier() with params []
AppendedHistGradientBoostingClassifier() with params []
AppendedKNeighborsClassifier() with params []
AppendedLabelPropagation() with params []
AppendedLabelSpreading() with params []
AppendedLinearDiscriminantAnalysis() with params []
AppendedLinearSVC() with params []
AppendedLogisticRegression() with params []
AppendedLogisticRegressionCV() with params []
AppendedMLPClassifier() with params []
AppendedMultinomialNB() with par

In [ ]:
# visualize plots by different model parameters
plt_models_by_clf = []
for model_name, model_class, model_params, model_score in insights:
    



In [ ]:
# pick best accuracy models and compare them to the rest
plt_model_best = []
_count = 0
for model_name, model_class, model_params, model_score in insights:
    if not model_name in plt_model_best:
        plt_model_best.append({"name": model_name, "parameters": model_params, "accuracy": model_score})
    else:
        for best_name, best_parameters, best_accuracy in plt_model_best:
            _count = _count + 1
            if best_name == model_name:
                if best_accuracy < model_score:
                    plt_model_best[_count]["parameters"] = model_params
                    plt_model_best[_count]["accuracy"] = model_score
                if best_accuracy == model_score:
                    plt_model_best[_count]["parameters"] = best_parameters + model_params

                    
